# Run This Notebook

<h3><a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/stories/derechos.ipynb&branch=main">🚀 Launch in VEDA JupyterHub (requires access)</a></h3>

<h4> To obtain credentials to VEDA Hub, <a href = "https://docs.openveda.cloud/user-guide/scientific-computing/getting-access.html"> follow this link for more information.</a></h4>

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed. </div>

<h4> If you <strong>do not</strong> have a VEDA Jupyterhub Account you can launch this notebook on your local environment using MyBinder by clicking the icon below.</h4>
<br/>
<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Fstories%2Fderechos.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Environment Setup

In [1]:
# Load libraries
#!pip install -q earthaccess pandas xarray fsspec requests pystac_client

import datetime
import glob
import os

import earthaccess
import pandas as pd
import plotutils as putils
import requests
import xarray as xr
from earthaccess import Store
from pystac_client import Client

In [2]:
# For retrieving data already catalogued in VEDA STAC
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

# What is a Derecho?

A **derecho** is a long-lasting, fast-moving windstorm that is produced from a line of severe thunderstorms. For a storm to be called a derecho, there must be:

- A concentrated area of severe wind reports over 400 miles (650 km) in length and at least 60 miles (96 km) wide.
- Several wind gusts reported over 75 mph (120 km/h).
- Sustained winds of at least 58 mph (93 km/h). (SPC)

Unlike single thunderstorms, derechos form within mesoscale (i.e., mid-sized) convective systems (MCS)—large storm clusters that produce strong, long-lasting straight-line winds.

## Conditions that Help a Derecho Form Include:

- **Strong instability** (Convective Available Potential Energy (CAPE) over 2000–4000 J/kg): provides energy for strong updrafts and intense thunderstorms.  
- **High low-level moisture** (dewpoints of 65–75 °F): keeps storms going by supplying moisture.  
- **Strong mid- and upper-level winds** (wind shear over 40 knots): help organize storms and push them forward.  
- **A well-defined cold pool**: rain-cooled air at the surface strengthens the storm by increasing wind speeds at the front of the system.  



# Example: Soil Moisture from GLDAS

Retrieve Global Land Data Assimilation System (GLDAS) data from the NASA EarthData portal. GLDAS data helps assess soil moisture levels before and after the storm, showing how pre-existing drought conditions contributed to dust transport and how heavy rainfall may have impacted runoff and flooding.

### Processing steps:
1.) Provide credentials for EarthAccess authentication<br />
2.) Select a start and end date for the data granules<br />
3.) Add <a href="https://search.earthdata.nasa.gov/search/granules?p=C1690022359-GES_DISC&pg[0][v]=f&pg[0][gsk]=-start_date&q=GLDAS_CLSM10_3H_2.1&tl=1347014600.484!5!!&lat=0.05719976537514171&long=-0.07464366545408296&zoom=3.0033333333333334"
        target="_blank" rel="noopener noreferrer"
         style="color: blue; text-decoration: none;">
      model name and version from EarthData portal
    </a><br />
4.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
5.) Retrieve all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
6.) Filter all collected granules based on the start_ and end_date range.<br />
7.) Compute daily mean of hourly files<br />
8.) Plot variable based on coordinates selected<br />
9.) Create a .gif over the start_ and end_date range

In [3]:
# ── Authenticate ──────────────────────────────────────────────────────────────
auth = earthaccess.login()

store = Store(auth)

## Select dates and find links

In [ ]:
# ── Search by date ────────(requires >2GB RAM if more than 40 dates are selected)───────────────
start_date = datetime.datetime(2022, 4, 1)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq="D").to_pydatetime()

# ── Dataset ─────────────────────────────────────────────────────────────────────────────────────
short_name = "GLDAS_NOAH025_3H"
version = "2.1"
variable = "SoilMoi0_10cm_inst"  # Only select a single variable of interest

# get all granules
print("Retrieving data granules from Earthaccess")
results = earthaccess.search_data(
    short_name=short_name,
    version=version,
    temporal=(start_date, end_date),
    cloud_hosted=True,
)

# grab the S3 URLs
urls = [g["umm"]["RelatedUrls"][1]["URL"] for g in results]
print(f"Found {len(urls)} files")

# ── Open granules ───────────────────────────────────────────────────────────────────────────────
granules = earthaccess.open(results)
fs = earthaccess.get_s3_filesystem(results=results)  # auto‐picks the right DAAC & creds

fsspec_caching = {"cache_type": "mmap"}
lat_range = slice(24, 50)
lon_range = slice(-125, -66)


def subset(ds):
    return ds[[variable]].sel(lat=lat_range, lon=lon_range)


ds = xr.open_mfdataset(
    [fs.open(url, **fsspec_caching) for url in urls],
    chunks="auto",
    concat_dim="time",
    combine="nested",
    parallel=True,
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="exact",
    preprocess=subset,
)

# ── Compute Daily Mean ───────────────────────────────────────────────────────────────────────────────
ds_resampled = ds.resample(time="1D").mean(dim="time").compute()
ds_resampled

Retrieving data granules from Earthaccess
Found 329 files


QUEUEING TASKS | :   0%|          | 0/329 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/329 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/329 [00:00<?, ?it/s]

## Create interactive map

In [ ]:
# Use the plot_folium_from_xarray function from plotutils
putils.plot_folium_from_xarray(
    dataset=ds_resampled["SoilMoi0_10cm_inst"],
    day_select="2022-05-11",
    bbox=[-130, 33, -90, 50],
    var_name_for_title="Soil Moisture from GLDAS (m³/m³) [subset]",
    matplot_ramp="YlOrRd_r",
    zoom_level=4.5,
    flipud=False,
    save_tif=False,
    tif_filename=None,
    crs="4326",
    opacity=0.8,
)

## Create an animation over dates

In [ ]:
# Usage:
putils.matplotlib_gif(
    data=ds_resampled["SoilMoi0_10cm_inst"],
    bbox=[-130, 33, -90, 50],
    gif_savename="soil_matplotlib.gif",
    duration=2,
    cmap="YlOrRd_r",
)

# Example: Soil moisture data from WLDAS
This example pulls soil moisture data from the Western Land Data Assimilation System (WLDAS) via the VEDA STAC catalog and visualize.The WLDAS is a regional instance of NASA’s Land Information System (LIS), developed at Goddard Space Flight Center for the western United States.  It integrates meteorological forcings (precipitation, radiation, temperature, humidity, wind, surface pressure) and satellite-derived parameters (vegetation class, soil texture, elevation) into the Noah-MP land surface model using data assimilation techniques.

Soil moisture critically controls the partitioning of net surface energy into latent (evapotranspiration) versus sensible (heating) fluxes. Wetter soils favor latent heat, stabilizing the boundary layer, whereas drier soils boost sensible heating, enhancing near-surface temperature and convective available potential energy (CAPE). These processes govern where and when thunderstorms can initiate and organize.



## Processing steps:
1.) Choose STAC catalog ID and date<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

## Choose variable and retrieve json from VEDA STAC catalogue

In [ ]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "wldas-derecho-sm"
date = "2022-05-11"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

## Retrieve tiling information

In [ ]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "hot"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

## Plot data

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.5, -110],
    zoom_level=5,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="WLDAS soil moisture",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Soil Moisture (m3/m3)",
    attribution="VEDA WLDAS Soil Moisture",
    tile_name="WLDAS Soil Moisture Data",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Example:  Hourly Aerosol Optical Thickness from MERRA-2

This example retrieves MERRA-2 hourly files for Aerosol Optical Thickness. Aerosol Optical Thickness (AOT), also called Aerosol Optical Depth (AOD), is a dimensionless measure of how much sunlight aerosols—tiny particles like dust, smoke or sea salt—scatter and absorb as it travels through a column of atmosphere. In practical terms, an AOT of 0.1 means only 10 % of the direct solar beam is extinguished by aerosols before reaching the surface.

The intense straight-line winds in derechos can uplift large quantities of soil and dust, dramatically increasing AOT downwind. Tracking AOT in near-real-time reveals the spatial extent and intensity of these airborne dust plumes.



## Processing steps:
1.) Select a start and end date for the data granules<br />
2.) Add <a href="https://search.earthdata.nasa.gov/search/granules/collection-details?p=C1276812830-GES_DISC&pg[0][v]=f&pg[0][gsk]=-start_date&q=merra%20aerosol&tl=1031468010.052!5!!"
        target="_blank" rel="noopener noreferrer"
         style="color: blue; text-decoration: none;">
      model name and version from EarthData portal
    </a><br />
3.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
4.) Retrive all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
5.) Filter all collected granules based on the start_ and end_date range.<br />
6.) Plot variable based on coordinates selected<br />
7.) Create a .gif over the start_ and end_date range

## Select and filter

In [ ]:
# ── Search by date ──────────────────────────────────────────────────────────────────────────────
start_date = datetime.datetime(2022, 5, 12)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq="D").to_pydatetime()

# ── Dataset ─────────────────────────────────────────────────────────────────────────────────────
short_name = "M2T1NXAER"
version = "5.12.4"
variable = "TOTEXTTAU"  # Total aerosol extinction optical thickness

# get all granules
print("Retrieving data granules from Earthaccess")
results = earthaccess.search_data(
    short_name=short_name,
    version=version,
    temporal=(start_date, end_date),
    cloud_hosted=True,
)

# grab the S3 URLs
urls = [g["umm"]["RelatedUrls"][1]["URL"] for g in results]
print(f"Found {len(urls)} files")

# ── Open granules ───────────────────────────────────────────────────────────────────────────────
granules = earthaccess.open(results)
fs = earthaccess.get_s3_filesystem(results=results)  # auto‐picks the right DAAC & creds

fsspec_caching = {"cache_type": "mmap"}
lat_range = slice(24, 50)
lon_range = slice(-125, -66)


def subset(ds):
    return ds[[variable]].sel(lat=lat_range, lon=lon_range)


ds = xr.open_mfdataset(
    [fs.open(url, **fsspec_caching) for url in urls],
    chunks="auto",
    concat_dim="time",
    combine="nested",
    parallel=True,
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="exact",
    preprocess=subset,
)

ds

## Plot time series of Aerosols

Open the MERRA file and plot by hour in a .gif for the selected variable AOT

In [ ]:
# Use the matplotlib_gif function from plotutils
putils.matplotlib_gif(
    data=ds["TOTEXTTAU"],
    bbox=[-130, 33, -90, 50],
    gif_savename="merra_aot.gif",
    duration=2,
    cmap="YlOrRd_r",
);

# Example: Aerosol Optical Depth with MODIS

Pull MODIS Aerosol Optical Depth data from the VEDA STAC catalog and visualize


In [ ]:
collection_id = "modis-derecho"
date = "2022-05-12"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

In [ ]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "viridis"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[42.5, -96],
    zoom_level=7,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    layer_name="MODIS AOD",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Aerosol Optical Depth",
    attribution="VEDA MODIS AOD",
    tile_name="MODIS AOD",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Example: Wind Gusts

This examples pulls data the National Centers for Environmental Information's (NCEI) Interpolated Wind Gusts data from the VEDA STAC catalog and visualizes. The NCEI Interpolated Wind Gusts product takes the discrete station‐reported peak wind gusts from the NCEI Storm Events Database—a standardized archive of severe‐weather observations dating back to 1950—and uses spatial interpolation to generate a continuous gridded field of maximum gust speeds across the derecho swath. 

By filling in the gaps between point measurements, it reveals the full geographic extent and intensity gradients of the derecho’s outflow winds, often uncovering zones of extreme gust (e.g., 70 mph +) that lie between and beyond individual station sites. 

In [ ]:
collection_id = "windgusts-derecho"
date = "2022-05-12"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

In [ ]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "viridis"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[45, -97.7],
    zoom_level=6,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Wind Gusts",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Wind Gusts (mph)",
    attribution="VEDA Wind Gusts",
    tile_name="Wind Gusts",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Example: Black Marble Night Lights

This examples pulls the NASA Black Marble Night Lights data from the VEDA STAC catalog and visualizes.
During extreme windstorms such as derechos, sudden drops in nighttime brightness in the Black Marble imagery reveal where power infrastructure has failed and outages have occurred, while the re-illumination of areas over subsequent days tracks the pace and extent of electrical service restoration. This makes Black Marble a powerful, near-real-time proxy for assessing societal impacts and grid resilience in the storm’s wake.

In [ ]:
collection_id = "nightlights-derecho"
date = "2022-05-10"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

In [ ]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "bwr"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[45, -93],
    zoom_level=8,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Black Marble Night Lights",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Artificial Light",
    attribution="VEDA Black Marble Night Lights",
    tile_name="Black Marble Night Lights",
    opacity=0.8,
    height="800px",
)

print(
    "Visualization of St. Paul, Minnesota where values approaching 255 indicate power outages."
)
# Display the map
m


# Example: Global Precipitation

Pull NASA's Global Precipitation Measurement (GPM) data through the Common Metadata Repository (CMR) STAC API.
NASA's GPM uses satellites to measure Earth's rain and snowfall for the benefit of mankind. Launched by NASA and JAXA on Feb. 27th, 2014, GPM is an international mission that sets the standard for spaceborne precipitation measurements.



In [ ]:
# Specific for May 05, 2022 (using titiler-cmr request)
date = "2022-05-12"
basetile_URL = "https://staging.openveda.cloud/api/titiler-cmr/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?"
tile_URL = f"datetime={date}T00%3A00%3A00.000Z%2F{date}T23%3A59%3A59.999Z&resampling=bilinear&variable=precipitation&colormap_name=gnbu&rescale=0%2C46&concept_id=C2723754864-GES_DISC&backend=xarray"
tilejson_url = f"{basetile_URL}{tile_URL}"

In [ ]:
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tilejson_url,
    layer_name="GPM Imerge Precipitation",
    colorbar_caption="mm/hr",
    date="2022-05-12",
    colormap_name="Blues",
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    center_coords=[46.55, -96.94],
    zoom_level=5,
    height="800px",
    rescale=(0, 46),
)

m

## Economic Impact
The economic impact reached beyond the crop fields. Many grain silos, irrigation systems, and farm buildings were also damaged or destroyed, adding to the financial burden on farmers.  See image below to identify crop types across the Midwest and the Storm Prediction Center reports across the Midwest.

### Compare Corn vs Soybean
Use the slider below to compare the USDA corn and soybean land cover for the year 2022 and all unfiltered storm reports collected by the Storm Prediction Center (SPC) on May 12, 2022. 

Image credit [USDA](https://www.nass.usda.gov/Research_and_Science/Cropland/sarsfaqs2.php) and [Storm Prediction Center](https://www.spc.noaa.gov/).

In [ ]:
# Usage example:
img1_path = "images/derecho/Derecho-Crop.jpg"
img2_path = "images/derecho/Derecho-storm-reports.jpg"

putils.create_pausable_blend_slider(img1_path, img2_path, width=800)

# Clean Up (Optional)


Remove any **.gif** files that were created to save on storage space. Additionally, remove any **core** files that were created if the kernel crashed.

In [ ]:
# find all .gif files in the current directory
for gif_path in glob.glob("*.gif"):
    try:
        os.remove(gif_path)
        print(f"Removed {gif_path}")
    except OSError as e:
        print(f"Error removing {gif_path}: {e}")

# find all core files in the current directory
for core_path in glob.glob("core.*"):
    try:
        os.remove(core_path)
        print(f"Removed {core_path}")
    except OSError as e:
        print(f"Error removing {core_path}: {e}")